In [1]:
import sys
sys.path.append('/MS3D')
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils
from pcdet.datasets import build_dataloader

cfg_file = '/MS3D/tools/cfgs/target_waymo/ms3d_nuscenes_voxel_rcnn_centerhead.yaml'

# Get target dataset
cfg_from_yaml_file(cfg_file, cfg)
logger = common_utils.create_logger('temp.txt', rank=cfg.LOCAL_RANK)
if cfg.get('DATA_CONFIG_TAR', False):
    dataset_cfg = cfg.DATA_CONFIG_TAR
    classes = cfg.DATA_CONFIG_TAR.CLASS_NAMES
    cfg.DATA_CONFIG_TAR.USE_PSEUDO_LABEL=False
    if dataset_cfg.get('USE_TTA', False):
        dataset_cfg.USE_TTA=False
else:
    dataset_cfg = cfg.DATA_CONFIG
    classes = cfg.CLASS_NAMES
dataset_cfg.DATA_SPLIT.test = 'train'
dataset_cfg.USE_CUSTOM_TRAIN_SCENES = True
dataset_cfg.SAMPLED_INTERVAL.test = 2

target_set, _, _ = build_dataloader(
            dataset_cfg=dataset_cfg,
            class_names=classes,
            batch_size=1, logger=logger, training=False, dist=False
        )


fatal: unsafe repository ('/MS3D' is owned by someone else)
To add an exception for this directory, call:

	git config --global --add safe.directory /MS3D


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2023-05-12 05:48:16,053   INFO  Loading Waymo dataset
2023-05-12 05:48:32,061   INFO  Total skipped info 0
2023-05-12 05:48:32,062   INFO  Total samples for Waymo dataset: 37680
2023-05-12 05:48:32,074   INFO  Total sampled samples for Waymo dataset: 18840


9420

In [2]:
from pcdet.utils import box_fusion_utils
from tqdm import tqdm 
import numpy as np

dets_txt = '/MS3D/tools/cfgs/target_waymo/raw_dets/train5hz_dets.txt'
print(f'classes: {cfg.DATA_CONFIG_TAR.CLASS_NAMES}')

ps_dict = {}
det_annos = box_fusion_utils.load_src_paths_txt(dets_txt)
combined_dets = box_fusion_utils.combine_box_pkls(det_annos, cfg.DATA_CONFIG_TAR.CLASS_NAMES)

classes: ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']


aggregating_proposals: 100%|████████████████████████████| 18840/18840 [00:30<00:00, 626.13it/s]


# Task:
1. Have diff discard/radius for diff classes
Maybe something like this?
```yaml
# Check that all lists are same length
classes: ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']
discard: [4,4,4, 4,4, 4]
radius: [2,2,2, 0.5,0.5, 0.3]
```
But if overlapping classes e.g. cyclist and pedestrian, you'd want the predominant class to be kept. Maybe it's okay to just match by class, then later on use NMS to keep the more confident classes?

2. Weighing by detection quantity

I wonder if there's some way to weigh by the number of detections it received? Low score but many detections make for a strong proposal.

#### Comments
Fine-tuning lyft -> waymo for ITSC submission was fine to use car,truck,bus -> vehicle,vehicle,vehicle. So I guess it doesn't affect the training to map it like this. I think it probably rendered the truck/bus heads useless since we labelled everything as 1 i.e. car. 

In [23]:
def conf_scaling_coeff(num_boxes, num_boxes_at_unity, eq_type='sqrt', max_scaling=2.0):
    """
    This function returns the scaling coefficient to scale the confidence score based on
    the number of boxes proposed for an object. Two scaling equations are provided.
    
    Note: This might be bad if many detectors detect but wrongly detect -> adjust max_scaling according
    
    num_boxes (list of ints): number of input boxes for each kbf box fusion
    num_boxes_at_unity (int): The point at which the scaling coeff is equal to 1. 
                        Less than this and coeff < 1; More than this and coeff > 1.
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (1/np.sqrt(num_boxes_at_unity)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (1/num_boxes_at_unity) * num_boxes
        
    else:
        raise notImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=0)

In [79]:
# ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']
classes=target_set.dataset_cfg.CLASS_NAMES
discard=[2,2,2,2,2,2]
radius=[2,2,2, 0.5,0.5, 0.3]
kbf_nms=[0.1,0.1,0.1,0.5,0.5,0.5,0.5]
min_score = 0.3

# confidence scaling
scale_conf_by_ndets = False
num_boxes_at_unity = 3
max_scale_score = cfg.SELF_TRAIN.SCORE_THRESH + 0.1

# Downsample
ds_combined_dets = combined_dets[::5]

# Get class specific config
cls_kbf_config = {}
for enum, cls in enumerate(classes):
    if cls in cls_kbf_config.keys():
        continue
    cls_kbf_config[cls] = {}
    cls_kbf_config[cls]['cls_id'] = enum+1 # in OpenPCDet, cls_ids enumerate from 1
    cls_kbf_config[cls]['discard'] = discard[enum]
    cls_kbf_config[cls]['radius'] = radius[enum]
    cls_kbf_config[cls]['nms'] = kbf_nms[enum]

for frame_boxes in tqdm(ds_combined_dets, total=len(ds_combined_dets)):
        
    boxes_lidar = np.hstack([frame_boxes['boxes_lidar'],
                             frame_boxes['class_ids'][...,np.newaxis],
                             frame_boxes['score'][...,np.newaxis]])
    
    boxes_names = frame_boxes['names']
    unique_classes = np.unique(boxes_names)    
    ps_label_nms = []
    for cls in unique_classes:
        cls_mask = (boxes_names == cls)
        cls_boxes = boxes_lidar[cls_mask]
        score_mask = cls_boxes[:,8] > min_score
        cls_kbf_boxes, num_dets_per_box = box_fusion_utils.label_fusion(cls_boxes, 'kde_fusion', 
                                       discard=cls_kbf_config[cls]['discard'], 
                                       radius=cls_kbf_config[cls]['radius'], 
                                       nms_thresh=cls_kbf_config[cls]['nms'], 
                                       weights=None)
        
        # E.g. car,truck,bus,motorcycle,bicycle,pedestrian (id: 1,2,3,4,5,6) -> Veh,Cyc,Ped (id: 1,4,6)
        cls_kbf_boxes[:,7] = cls_kbf_config[cls]['cls_id']
        
        # Scale confidence score by number of detections
        if scale_conf_by_ndets:
            coeff = conf_scaling_coeff(num_dets_per_box, num_boxes_at_unity)
            mask = cls_kbf_boxes[:,8] < cfg.SELF_TRAIN.SCORE_THRESH
            cls_kbf_boxes[mask,8] *= coeff[mask]
            cls_kbf_boxes[mask,8]= np.clip(cls_kbf_boxes[mask,8], 0, max_scale_score)
            
        ps_label_nms.extend(cls_kbf_boxes)
        
    ps_label_nms = np.array(ps_label_nms)
    
    # neg_th < score < pos_th: ignore for training but keep for update step
    pred_boxes = ps_label_nms[:,:7]
    pred_labels = ps_label_nms[:,7]
    pred_scores = ps_label_nms[:,8]
    ignore_mask = pred_scores < cfg.SELF_TRAIN.SCORE_THRESH
    pred_labels[ignore_mask] = -pred_labels[ignore_mask]
    gt_box = np.concatenate((pred_boxes,
                            pred_labels.reshape(-1, 1),
                            pred_scores.reshape(-1, 1)), axis=1)    

    gt_infos = {
        'gt_boxes': gt_box,
    }
    ps_dict[frame_boxes['frame_id']] = gt_infos

100%|██████████████████████████████████████████████████████| 3768/3768 [09:54<00:00,  6.34it/s]


In [110]:
import copy
from tqdm import tqdm

dataset = target_set

pred_annos, gt_annos = [], []
eval_gt_annos = copy.deepcopy(dataset.infos)
for frame_id in tqdm(ps_dict.keys(), total=len(ps_dict.keys())):  
    boxes = ps_dict[frame_id]['gt_boxes'].copy()            
    boxes[:,:3] -= dataset.dataset_cfg.SHIFT_COOR # just for eval in this notebook
    p_anno = {"frame_id": frame_id,
              "name": np.array([dataset.dataset_cfg.CLASS_NAMES[int(abs(box[7]))-1] for box in boxes]),
              "pred_labels": np.ones(len(boxes)),
              "boxes_lidar": boxes[:,:7],
              "score": boxes[:,8]}
    pred_annos.append(p_anno)
    gt_annos.append(eval_gt_annos[dataset.frameid_to_idx[frame_id]]['annos'])

100%|███████████████████████████████████████████████████| 4220/4220 [00:00<00:00, 17627.72it/s]


In [ ]:
# ap_result_str, ap_dict = dataset.kitti_eval(pred_annos, eval_gt_annos, class_names=['Vehicle'])
# ap_result_str, ap_dict = dataset.evaluation(pred_annos, dataset.dataset_cfg.CLASS_NAMES, eval_metric='waymo')
from pcdet.datasets.waymo.waymo_eval import OpenPCDetWaymoDetectionMetricsEstimator
eval = OpenPCDetWaymoDetectionMetricsEstimator()

ap_dict = eval.waymo_evaluation(
    pred_annos, gt_annos, class_name=dataset.dataset_cfg.CLASS_NAMES,
    distance_thresh=1000, fake_gt_infos=dataset.dataset_cfg.get('INFO_WITH_FAKELIDAR', False)
)

Start the waymo evaluation...
Number: (pd, 244696) VS. (gt, 353781)
Level 1: 293563, Level2: 60218
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.


2023-05-12 07:03:58.780081: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-12 07:03:58.782075: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-12 07:03:58.782879: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2023-05-12 07:03:58.783758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
202

#### Scaling by number of detectors

This desmos grapher was useful for coming up with below https://www.desmos.com/calculator

At the end of the day we just wanted to map one range to another in a non-linear fashion where we can specify how many detections would give a scaling=1; or what the max scaling is.

In [20]:
import matplotlib.pyplot as plt

# Set a config where you can enable/disable this
def min_boxes_conf_scaling(num_boxes, min_boxes, eq_type='sqrt', max_scaling=2.0):
    """
    For this function, at min_boxes, scaling is 1. We clip if it gets too large.    
    Adjustable param: min_boxes
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (1/np.sqrt(min_boxes)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (1/min_boxes) * num_boxes
        
    else:
        raise notImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=0)


def total_dets_conf_scaling(num_boxes, total_num_detectors, eq_type='sqrt', max_scaling=2.0):
    """
    For this function, at total_num_detectors, scaling is max_scaling. The point where scaling=1
    will change according to the total_num_detectors which may not be optimal.
    
    Adjustable param: total_num_detectors
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (max_scaling/np.sqrt(total_num_detectors)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (max_scaling/total_num_detectors) * num_boxes
        
    else:
        raise notImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=0)


# min_dets = 20
# x = np.arange(0,30)
# # y = (1/np.sqrt(min_dets)) * np.sqrt(x) 
# y = 1/20 * x
# plt.plot(x,y)
# plt.grid(True)

# Non-linear scaling with sqrt(x)
# using 2/sqrt(num_det) means that at num_dets, scaling is max_scaling -> But we don't set min_dets
# Adjustable param: max_scaling
# num_detectors = 24
# max_scaling = 2
# x = np.arange(0,30)
# y = (max_scaling/np.sqrt(num_detectors)) * np.sqrt(x) 
# plt.plot(x,y)
# plt.grid(True)
scale1 = total_dets_conf_scaling(1, 6, eq_type='sqrt') # if total detectors is low, then the scaling is jumpy
scale2 = min_boxes_conf_scaling(6, 3, eq_type='sqrt') # works better for varying min_boxes
scale1, scale2

(0.8164965809277261, 1.4142135623730951)